In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [11]:
### Load the trained model, scaler pickle , onehot
model = load_model('model.h5')

## load the encoder and scaler
#'rb' -> read byte mode beacuse pickling is a deserialized format(like we can serializeit , we can desrialize it)
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)    

In [12]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [13]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

e:\UDemy Final\ANN Classification\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
# here input_data is in key-value pair and we need to convert it into dataframe first
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [16]:
## Encode categorical variables gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

# (I've already done one hot encoding for geographic column i.e "geo_encoded_df")

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
##  concatenation one hot encoded data 

# combine one-hot encoded ( geography columns) with input data (& we have already added label_encoder gender data into "input_df")
input_df = pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df], axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

After scaling , all these values has basically got scaled and get all the data in the form of array...now finally we need to just predict and see whether the person is going to leave the bank or not

In [19]:
# Predict churn
prediction =model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 910ms/step


array([[0.05440924]], dtype=float32)

In [20]:
prediction_proba = prediction[0][0]
prediction_proba

0.05440924

In [21]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")    

The customer is not likely to churn.
